In [ ]:
import nltk
import sys
nltk.download('wordnet')
!pip install -U sentence-transformers
sys.path.append('/content/drive/MyDrive/')

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
from utils import *
import pickle

model = SentenceTransformer('all-mpnet-base-v2', device='cuda')
data, data_train, data_seen_500, data_unseen_500, data_test = load_data()

In [ ]:
X = []
y = []
for word, defi_set in data_train.items():
    for defi in defi_set:
        X.append(defi)
        y.append(word)

In [ ]:
X_embeddings = model.encode(X, device='cuda', show_progress_bar=True)

Batches:   0%|          | 0/22327 [00:00<?, ?it/s]

In [ ]:
with open('/content/drive/MyDrive/untrained_embeds.pkl', 'wb') as f:
    pickle.dump({
        'X_embeddings': X_embeddings,
        'y': y
    }, f)

### Fit training data

In [ ]:
# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_embeddings)

NearestNeighbors(metric='cosine')

### Test seen data

In [ ]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_seen_500.items()): # <--- use seen_500 data here
    query = list(defi)[0]
    query = model.encode(query)

    prediction = search([query], knn, y, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 492/492 [18:18<00:00,  2.23s/it]

acc@1: 0.87
acc@10: 1.0
acc@100: 1.0
median rank: 0
standard error of mean rank: 0


(0.8678861788617886, 1.0, 1.0, 0.0, 0.01850262528637175)

### Test unseen data

In [13]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_unseen_500.items()): # <--- use seen_500 data here
    query = list(defi)[0]
    query = model.encode(query)

    prediction = search([query], knn, y, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 481/481 [17:44<00:00,  2.21s/it]

acc@1: 0.29
acc@10: 0.63
acc@100: 0.83
median rank: 3
standard error of mean rank: 13


(0.29313929313929316,
 0.632016632016632,
 0.8253638253638254,
 3.0,
 12.95363978474621)

### Test description set (200) data

In [15]:
X = []
y = []
for word, defi_set in data.items(): # <--- fit all data here
    for defi in defi_set:
        X.append(defi)
        y.append(word)

X_embeddings = model.encode(X, device='cuda', show_progress_bar=True)

knn = NearestNeighbors(metric='cosine')
knn.fit(X_embeddings)

y_pred = []
y_gold = []
for word, defi in tqdm(data_test.items()): # <--- use desc_200 data here
    query = list(defi)[0]
    query = model.encode(query)

    prediction = search([query], knn, y, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

Batches:   0%|          | 0/22349 [00:00<?, ?it/s]

100%|██████████| 200/200 [06:47<00:00,  2.04s/it]

acc@1: 0.26
acc@10: 0.56
acc@100: 0.89
median rank: 6
standard error of mean rank: 15


(0.255, 0.56, 0.885, 6.0, 14.761464015300108)